In [3]:
import pandas as pd
import requests
from datetime import datetime
import pytz

In [1]:
#pip install --upgrade requests 

In [5]:
# retrieve the data as a .json file from the weather website - this returns a list of data

city = 'Berlin'

API_key = 'Your API KEY'

url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

r = requests.get(url)
json = r.json()

#json

In [6]:
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
r = requests.get(url)
json = r.json()

In [7]:
###
# to get a timestamp showing when the data was retieved 
# Datetime uses the uses the current time of the system, - on local computers this is normally correct 
# in the cloud, computers are not always in the same country, therefore the timezone module is required 
# to ensure that the timestamp is local to user and not the computer

tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

#now

In [9]:
# create a dictionary to store the required weather data 
# this in turn facilitates the creation of the dataframe

weather_dict ={'city':[],
               'country': [],
               'forecast_time': [],
               'outlook': [],
               'detailed_outlook': [],
               'temperature': [],
               'temperature_feels_like': [],
               'clouds': [],
               'rain': [],
               'snow': [],
               'wind_speed': [],
               'wind_deg': [],
               'humidity': [],
               'pressure': [],
               'information_retrieved_at': []
              }

# use a for loop to extract the data and append them to the dictionary lists

for i in json['list']:
    weather_dict['city'].append(json['city']['name'])
    weather_dict['country'].append(json['city']['country'])
    weather_dict['forecast_time'].append(i['dt_txt'])
    weather_dict['outlook'].append(i['weather'][0]['main'])    
    weather_dict['detailed_outlook'].append(i['weather'][0]['description'])    
    weather_dict['temperature'].append(i['main']['temp']) 
    weather_dict['temperature_feels_like'].append(i['main']['feels_like'])    
    weather_dict['clouds'].append(i['clouds']['all'])    
    # because some data will not always be present an exception process is needed
    # for example if there's no rain we return 'dry'
    try:
        weather_dict['rain'].append(i['rain']['3h'])
    except:
        weather_dict['rain'].append('dry')
    try:
        weather_dict['snow'].append(i['snow']['3h'])
    except:
        weather_dict['snow'].append('none')       
    weather_dict['wind_speed'].append(i['wind']['speed'])    
    weather_dict['wind_deg'].append(i['wind']['deg'])    
    weather_dict['humidity'].append(i['main']['humidity'])
    weather_dict['pressure'].append(i['main']['pressure'])    
    weather_dict['information_retrieved_at'].append(now.strftime('%a %d/%m/%Y %H:%M:%S'))        

In [10]:
# after the data has been appended in theh dictionary, convert to a dataframe

weather_df = pd.DataFrame(weather_dict)

#weather_df.head()

In [11]:
# combine the process so far in a function that accepts a list of cities as input
# firstly set up all the variables used

def weather_loop(cities):

    API_key = 'Your API KEY'

    tz = pytz.timezone('Europe/Berlin')
    now = datetime.now().astimezone(tz)

    weather_dict = {
        'city': [],
        'country': [],
        'forecast_time': [],
        'outlook': [],
        'detailed_outlook': [],
        'temperature': [],
        'temperature_feels_like': [],
        'clouds': [],
        'rain': [],
        'snow': [],
        'wind_speed': [],
        'wind_deg': [],
        'humidity': [],
        'pressure': [],
        'information_retrieved_at': []
    }

    for city in cities:

        url = (
            f'http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric'
        )
        r = requests.get(url)
        json = r.json()

        for i in json['list']:
            weather_dict['city'].append(json['city']['name'])
            weather_dict['country'].append(json['city']['country'])
            weather_dict['forecast_time'].append(i['dt_txt'])
            weather_dict['outlook'].append(i['weather'][0]['main'])
            weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
            weather_dict['temperature'].append(i['main']['temp'])
            weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
            weather_dict['clouds'].append(i['clouds']['all'])
            try:
                weather_dict['rain'].append(i['rain']['3h'])
            except:
                weather_dict['rain'].append('dry')
            try:
                weather_dict['snow'].append(i['snow']['3h'])
            except:
                weather_dict['snow'].append('none')
            weather_dict['wind_speed'].append(i['wind']['speed'])
            weather_dict['wind_deg'].append(i['wind']['deg'])
            weather_dict['humidity'].append(i['main']['humidity'])
            weather_dict['pressure'].append(i['main']['pressure'])
            weather_dict['information_retrieved_at'].append(
                now.strftime('%a %d/%m/%Y %H:%M:%S'))
        weather = pd.DataFrame(weather_dict)
            
    return weather #pd.DataFrame(weather_dict)

In [12]:
# test the function with named cities

weather_loop(['Berlin', 'Paris'])

## Solution using .json_normalise

In [13]:
# first call data using API as previously

city = 'Berlin'

API_key = 'Your API KEY'

url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

r = requests.get(url)
json = r.json()

In [63]:
# if .json_normalise is used on the list it will return nested lists under one col
# see weather col below, the nested lists need to be flattened out

pd.json_normalize(json['list']).head()

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h
0,1657303200,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-07-08 18:00:00,20.08,19.87,18.68,20.08,1016,1016,1019,66,1.40,75,4.36,268,8.28,d,NaN
1,1657314000,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-07-08 21:00:00,18.90,18.65,16.54,18.90,1019,1019,1019,69,2.36,63,4.20,278,9.26,n,NaN
2,1657324800,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-07-09 00:00:00,16.02,15.66,13.99,16.02,1021,1021,1019,76,2.03,62,3.83,287,9.48,n,NaN
3,1657335600,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-07-09 03:00:00,11.94,11.57,11.94,11.94,1023,1023,1018,91,0.00,53,3.61,285,9.61,d,NaN
4,1657346400,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000,0.0,2022-07-09 06:00:00,13.62,13.23,13.62,13.62,1023,1023,1018,84,0.00,74,3.71,269,8.25,d,NaN


In [14]:
# to flatten out nested lists parameters need to be added
# record path - col with nested lists
# meta = [] - provide the unnested col names
# errors - deals with missing values by inserting NaN

weather_json_df = pd.json_normalize(json['list'],
                                    record_path=['weather'],
                                    meta=['dt_txt', ['main', 'temp'],
                                        ['main', 'feels_like'],
                                        ['clouds', 'all'], ['rain', '3h'],
                                        ['snow', '3h'], ['wind', 'speed'],
                                        ['wind', 'deg'], ['main', 'humidity'],
                                        ['main', 'pressure']
                                    ],
                                    errors='ignore')

weather_json_df.head()

,id,main,description,icon,dt_txt,main.temp,main.feels_like,clouds.all,rain.3h,snow.3h,wind.speed,wind.deg,main.humidity,main.pressure
0,801,Clouds,few clouds,02d,2022-07-17 12:00:00,19.99,19.4,20,NaN,NaN,3.65,301,52,1016
1,801,Clouds,few clouds,02d,2022-07-17 15:00:00,21.45,20.83,24,NaN,NaN,3.24,309,45,1019
2,802,Clouds,scattered clouds,03d,2022-07-17 18:00:00,21.93,21.3,26,NaN,NaN,2.36,321,43,1021
3,803,Clouds,broken clouds,04n,2022-07-17 21:00:00,18.14,17.5,79,NaN,NaN,2.45,45,57,1025
4,803,Clouds,broken clouds,04n,2022-07-18 00:00:00,16.65,16.02,79,NaN,NaN,1.14,75,63,1025


In [15]:
# tidy the dataframe by removing unwanted columns 
# rename columns & add new columns

weather_json_df = weather_json_df.drop(columns=['id','icon']).copy()

In [16]:
weather_json_df.rename(columns={'main':'outlook',
                               'description':'detailed_outlook',
                               'dt_txt': 'forecast_time',
                               'main.temp': 'temperature',
                               'main.feels_like': 'temperature_feels_like',
                               'clouds.all': 'clouds',
                               'rain.3h': 'rain',
                               'snow.3h': 'snow',
                               'wind.speed': 'wind_speed',
                               'wind.deg': 'wind_deg',
                               'main.humidity': 'humidity',
                               'main.pressure': 'pressure',
                               },
                    inplace=True
                      )

# added cols

weather_json_df.insert(0, 'city',json['city']['name'])
weather_json_df.insert(1, 'country',json['city']['country'])
weather_json_df['last_update_at']= now.strftime('%d/%m/%Y %H:%M:%S')

In [17]:
# rearrange cols

weather_json_df = weather_json_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
       'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
       'wind_speed', 'wind_deg', 'humidity', 'pressure',
       'last_update_at']]

#weather_json_df.head()

In [18]:
# read the city dataframe to get the names of all cities we want weather data for

city_data = pd.read_csv("city_data.csv")
city_data.drop(columns = ["Unnamed: 0"], inplace = True)
#city_data.head()

In [25]:
# convert the city column to a list so it can be used as an input of the function call

cities = city_data["city"].to_list()

In [26]:
# create a function to capture the full process 

def get_weather_norm(cities):
    
    API_key = 'Your API KEY'
    df_list = []
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q=Berlin&appid={API_key}&units=metric") 
    test = requests.get(url)
    if test.status_code >= 200 and test.status_code <= 299:
        for city in cities:  
            url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric") 
            response = requests.get(url)
            json = response.json()
            json_norm_df = pd.json_normalize(json['list'],
                                             record_path=['weather'],
                                             meta=['dt_txt',
                                                   ['main', 'temp'],
                                                   ['main', 'feels_like'],
                                                   ['clouds', 'all'],
                                                   ['rain', '3h'],
                                                   ['snow', '3h'],
                                                   ['wind', 'speed'],
                                                   ['wind', 'deg'],
                                                   ['main', 'humidity'],
                                                   ['main', 'pressure']], 
                                             errors='ignore')
            json_norm_df['city'] = city
            json_norm_df.drop(columns=['id', 'icon'], inplace=True)
            json_norm_df.rename(columns={'main': 'outlook',
                                         'description': 'detailed_outlook',
                                         'dt_txt': 'forecast_time',
                                         'main.temp': 'temperature',
                                         'main.feels_like': 'temperature_feels_like',
                                         'clouds.all': 'clouds',
                                         'rain.3h': 'rain',
                                         'snow.3h': 'snow',
                                         'wind.speed': 'wind_speed',
                                         'wind.deg': 'wind_deg',
                                         'main.humidity': 'humidity',
                                         'main.pressure': 'pressure',},
                                inplace=True)
            #json_norm_df.insert(0, 'city', json['city']['name'])
            json_norm_df.insert(1, 'country', json['city']['country'])
            json_norm_df['information_retrieved_at'] = now.strftime("%d/%m/%Y %H:%M:%S")
            json_norm_df[['city', 'country', 'forecast_time', 'outlook', 'detailed_outlook',
                          'temperature', 'temperature_feels_like', 'clouds', 'rain', 'snow',
                          'wind_speed', 'wind_deg', 'humidity', 'pressure',
                          'information_retrieved_at']]
            df_list.append(json_norm_df)
    else:
        return -1
    weather_output = pd.concat(df_list, ignore_index=True)
    return weather_output      

In [23]:
# test using manual inputs

#get_weather_norm(['Cork', 'London']) 

In [27]:
# test using the cities_list as the input

weather_data = get_weather_norm(cities) 
#weather_data

In [28]:
# keep only the columns required

weather_data = weather_data[["detailed_outlook", "forecast_time", "temperature", "temperature_feels_like", "clouds", "rain", 
                             "wind_speed", "snow", "humidity","city"]]

In [29]:
# add the city_id column from imported city_data to the weather dataframe which will be used as the foreign key in MySQL

weather_data = weather_data.merge(city_data[["city_id", "city"]], how = "left", on = "city")
weather_data.rename(columns = {'detailed_outlook':'outlook', 'temperature_feels_like':'temp_feels_like'},inplace=True)
#weather_data.drop(['city_id_x','city_id_y'],axis=1,inplace=True)

In [30]:
# view the final table that should be output

weather_data.head()

,outlook,forecast_time,temperature,temp_feels_like,clouds,rain,wind_speed,snow,humidity,city,city_id
0,few clouds,2022-07-17 12:00:00,20.41,19.84,20,NaN,3.65,NaN,51,Berlin,Q64
1,few clouds,2022-07-17 15:00:00,21.73,21.11,24,NaN,3.24,NaN,44,Berlin,Q64
2,scattered clouds,2022-07-17 18:00:00,22.07,21.46,26,NaN,2.36,NaN,43,Berlin,Q64
3,broken clouds,2022-07-17 21:00:00,18.14,17.5,79,NaN,2.45,NaN,57,Berlin,Q64
4,broken clouds,2022-07-18 00:00:00,16.65,16.02,79,NaN,1.14,NaN,63,Berlin,Q64


In [31]:
# ensure weather types are correct

weather_data = weather_data.astype({'temperature':float,'temp_feels_like':float,'clouds':float,'rain':float,'wind_speed':float,'snow':float,'humidity':float,})
weather_data["forecast_time"] = pd.to_datetime(weather_data["forecast_time"])

In [32]:
weather_data.dtypes

outlook                    object
forecast_time      datetime64[ns]
temperature               float64
temp_feels_like           float64
clouds                    float64
rain                      float64
wind_speed                float64
snow                      float64
humidity                  float64
city                       object
city_id                    object
dtype: object

In [87]:
weather_data.to_csv("weather_data.csv")